In [ ]:
import pandas as pd
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# load recipe dataset
df = pd.read_csv("src/api/dataset/recipe_dataset.csv")

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.head(5)

In [ ]:
# df["ingredients"] = df["ingredients_list"].apply(lambda x: " ".join(ast.literal_eval(x)))
df["ingredients"] = df["ingredients_list"].apply(lambda x: sorted(eval(x)))
df["ingredients"] = df["ingredients"].apply(lambda x: [*set(x)])
df["ingredients"] = df["ingredients"].apply(lambda x: " ".join(x))

In [ ]:
df.head(5)

In [ ]:
df.iloc[2][2]

In [ ]:
v = TfidfVectorizer()
v.fit(df['ingredients'])
recipe_v = v.transform(df['ingredients'])

In [ ]:
ingredients = "courgette penne egg single cream parmesan cheese back bacon courgette flower"
test_data = v.transform([ingredients])

In [ ]:
cos_sim = map(lambda x: cosine_similarity(test_data, x), recipe_v)
scores = list(cos_sim)

In [ ]:
scores

In [ ]:
def rec_recipe(scores):
    top_five = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:10]
    rec_recipe = pd.DataFrame(columns = ['recipe', 'ingredient_measurements', 'score', 'url', 'ingredients'])
    count = 0
    for i in top_five:
        rec_recipe.at[count, 'recipe'] = df['recipe_name'][i]
        
        rec_recipe.at[count, 'ingredients'] = df['ingredients_measurements'][i]
        
        rec_recipe.at[count, 'url'] = df['recipe_urls'][i]
        rec_recipe.at[count, 'score'] = "{:.3f}".format(float(scores[i]))
        rec_recipe.at[count, 'ingredients'] = df['ingredients'][i]
        
        count += 1
    return rec_recipe

In [ ]:
tf = rec_recipe(scores)
tf

In [ ]:
tf_list = tf['ingredients'].to_list()
tf_list

In [ ]:
ingre_set = set(ingredients.split(" "))
rec_index = 0
for i, ingre in enumerate(tf_list):
    rec_set = set(ingre.split(" "))
    common_ingre = ingre_set.intersection(rec_set)
    if len(common_ingre) >= len(ingre_set):
        rec_index = i
        print(common_ingre)
        break
print(rec_index)

In [ ]:
ingre_set

In [ ]:
common_ingre

In [ ]:
tf.iloc[rec_index].to_dict()